In [1]:
from langame import LangameClient
import json
import openai
import datetime
from langame.conversation_starters import get_existing_conversation_starters
import logging
import os
from datasets import load_dataset
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, AutoTokenizer, DataCollatorForLanguageModeling, DataCollatorForLanguageModeling

c = LangameClient("../config.yaml")
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("classification")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 0ZcPZDcvuqgialj2wHpL, garbage
Skipping id: 0m2ltwf7wkxLzcHnFkQE, garbage
Skipping id: 1YWuwIp4srffwPNq7Aw9, garbage
Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: 4sIjdMJr0exliT9yfvuL, garbage
Skipping id: 5MAMBAYySyu3lKrR3zLt, garbage
Skipping id: 6jNOw0e7VA6lHWddtqMy, garbage
Skipping id: 7yHXwGPlHpopQAHU3xf1, garbage
Skipping id: 8e9jVhvFpUIzj7ocNv06, garbage
Skipping id: 8tzYKdv2Hkwxsm8eNQgj, garbage
Skipping id: 9NdMRn8C285Eu3k4PdsS, garbage
Skipping id: 9ZfEEkrB3cjLooW6YZM2, garbage
Skipping id: B8hRGbGGOweqEyxuuplE, garbage
Skipping id: BH9ZtGe1g6q3KmK7oV2f, garbage
Skipping id: BZjXMPPKlmcqXG23i0mI, garbage
Skipping id: BwSIpHO7X8N1fhf5zTyr, garbage
Skipping id: C0yrKrOeX5kQKwEgyrJS, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: D8A4QcAlqidVArVPDKGP, garbage
Skipping id: DhuuWoHv7oGKSA5agAzZ, garbage
Skipping id: DjuEuISOCOG6R2D7ZlSy, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: EBZCFk0aZPVS1SinJwcz, garbage
Skipping id

Using 8 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 11/06/2022, 14:30:50
There are 3059 embeddings of dim 768
	Compute estimated construction time of the index 11/06/2022, 14:30:50
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0002 secs
	Checking that your have enough memory available to create the index 11/06/2022, 14:30:50
11.2MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0016 secs
	Selecting most promising index types given data characteristics 11/06/2022, 14:30:50
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 11/06/2022, 14:30:50
		-> Instanciate the index HNSW15 11/06/2022, 14:30:50
		>>> Finished "-> Instanciate the index HNSW15" in 0.004

100%|██████████| 1/1 [00:00<00:00, 235.34it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0941 secs
	>>> Finished "Creating the index" in 0.0996 secs
	Computing best hyperparameters 11/06/2022, 14:30:50


	>>> Finished "Computing best hyperparameters" in 4.0102 secs
The best hyperparameters are: efSearch=8885
	Compute fast metrics 11/06/2022, 14:30:54
1052
	>>> Finished "Compute fast metrics" in 10.0376 secs
	Saving the index on local disk 11/06/2022, 14:31:04
	>>> Finished "Saving the index on local disk" in 0.0251 secs
Recap:
{'99p_search_speed_ms': 43.29755142000498,
 'avg_search_speed_ms': 9.505365207224562,
 'compression ratio': 0.9576690276384732,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=8885',
 'nb vectors': 3059,
 'reconstruction error %': 0.0,
 'size in bytes': 9812626,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 14.1983 secs


In [2]:
dataset = [(),()]

for e in memes:
  p = f"{','.join(e['topics'])} ### {e['content']}"
  dataset[0] += (p,)
  dataset[1] += (100.0,)
for e in c._firestore_client.collection("deleted_memes").stream():
  # check has "topics" and "content"
  if "topics" not in e.to_dict() or "content" not in e.to_dict():
    continue
  e = e.to_dict()
  p = f"{','.join(e['topics'])} ### {e['content']}"
  dataset[0] += (p,)
  dataset[1] += (1.0,)


In [1]:
import trlx
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# Steer a model with a collection of rated samples
model = trlx.train('distilgpt2', dataset=dataset)

NOTE: Redirects are currently not supported in Windows or MacOs.


AttributeError: module 'trlx' has no attribute 'train'

In [ ]:
# model is a wrapper with some logit preprocessing
model.generate(**tokenizer('Q: Who rules the world? A:', return_tensors='pt'), do_sample=True)